In [1]:
import requests
from bs4 import *
import argparse
import os
import pandas as pd

In [2]:
# add argument parser for passing the target folder for saving the images

parser = argparse.ArgumentParser()
parser.add_argument('--saveto', help='Target directory to save the Images (default: images/', action='store', dest='dirName')
parser.add_argument('--pages', help='No. of pages (default: 40', action='store', dest='last_pagination')
args = parser.parse_args()

In [3]:
if(args.dirName):

    dirName = str(args.dirName) + '/'

else :

    dirName = 'images/'

if(args.last_pagination):

    last_pagination = int(args.last_pagination) + 1

else :

    last_pagination = 40

if not os.path.exists(dirName):

    os.mkdir(dirName)

In [4]:
headers_std = {
'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.80 Safari/537.36',
'Content-Type': 'text/html',
}

In [5]:
link = input('Search Query URL: ')
req = requests.get(link)
soup = BeautifulSoup(req.text, 'lxml')

In [6]:
item_category = input('Enter the search category: ')


In [ ]:
urls = []

for x in range (1, last_pagination):
    urls.append(link + '&page=' + str(x))

img_urls = set()

k = 1 
pag = 1
for x in urls:
    print('Pagination : ' + str(pag))
    req = requests.get(x,headers=headers_std).text
    soup = BeautifulSoup(req, 'lxml')
    imgs = soup.find_all('img',{'class':'s-image'})
   
    for i in imgs:
        if str(i).find('src') != -1:
            url = i['src']
            img_urls.add(url)
            # name_image_folder = dirName + str(k) + '.jpg'
            # image = requests.get(url).content
            
            # with open(name_image_folder, 'wb') as handler:
            #     handler.write(image)
        k += 1
    pag += 1


In [7]:
urls =  list(img_urls)
categories = [item_category] * len(urls)

try:
    df1 = pd.read_csv('data.csv')
except:
    df1 = pd.DataFrame()

df2 = pd.DataFrame({'url' : urls, 'category':categories})



In [8]:
# concat both the dataframes
df = pd.concat([df1, df2])
print(len(df1))
print(len(df2))
print(len(df))
df.to_csv('data.csv',index=False)


In [9]:
import pandas as pd

df = pd.read_csv("data.csv",error_bad_lines=False)
df['image'] = df.apply(lambda row: str(row['id']) + ".jpg", axis=1)
df = df.sample(frac=1).reset_index(drop=True)
df.head(10)

In [10]:
from keras_preprocessing.image import ImageDataGenerator

image_generator = ImageDataGenerator(
    validation_split=0.2
)

training_generator = image_generator.flow_from_dataframe(
    dataframe=df,
    directory="images",
    x_col="image",
    y_col="subCategory",
    target_size=(60,80),
    batch_size=batch_size,
    subset="training"
)

In [11]:
validation_generator = image_generator.flow_from_dataframe(
    dataframe=df,
    directory="images",
    x_col="image",
    y_col="subCategory",
    target_size=(60,80),
    batch_size=batch_size,
    subset="validation"
)

In [12]:
import pickle
classes = len(training_generator.class_indices)
my_dict = training_generator.class_indices
key_list = list(my_dict.keys()) 
val_list = list(my_dict.values())
filename1 = 'key_list_sub'
filename = 'val_list_sub'
pickle.dump(key_list, open(filename1, 'wb'))
pickle.dump(val_list, open(filename, 'wb'))


In [13]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [14]:
#Initializing CNN
classifier = Sequential()

#1Convolution
classifier.add(Conv2D(32,(3,3),input_shape = (60,80,3), activation = 'relu'))

#2Pooling
classifier.add(MaxPooling2D(pool_size=(3, 3)))


In [15]:
#adding 2nd 3rd and 4th convolution layer
classifier.add(Conv2D(32,(3,3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(3, 3)))


#3Flattening
classifier.add(Flatten())


In [16]:
#4Full_Connection

classifier.add(Dense(units=32,activation = 'relu'))
classifier.add(Dense(units=64,activation = 'relu'))

classifier.add(Dense(units=128,activation = 'relu'))
classifier.add(Dense(units=256,activation = 'relu'))
classifier.add(Dense(units=256,activation = 'relu'))

classifier.add(Dense(units=classes,activation = 'softmax'))

In [17]:
#Compiling CNN
classifier.compile(optimizer='adam',
              loss="categorical_crossentropy",
              metrics=['accuracy'])

#classifier.summary()
#classifier.fit(train_images, train_labels, epochs=25, batch_size=100)


In [18]:
classifier.save("modelsubCategory.h5")
import numpy as np
from keras.preprocessing import image
filename = "40826.jpg"
from keras.models import load_model
new_model = load_model('model.h5')
new_model.summary()
test_image = image.load_img('images\\'+filename,target_size=(60,80))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = new_model.predict(test_image)
val = np.argmax(result)
my_dict = training_generator.class_indices
key_list = list(my_dict.keys()) 
val_list = list(my_di

In [19]:
print(key_list[val])

In [20]:
import pickle
# save the model to disk
filename1 = 'key_list'
filename = 'val_list'
pickle.dump(key_list, open(filename1, 'wb'))
pickle.dump(val_list, open(filename, 'wb'))